# 定义模型客户端

In [10]:
from openai import OpenAI

base_url = "http://127.0.0.1:8000/v1/"
client = OpenAI(api_key="EMPTY", base_url=base_url)

messages = [
    {"role": "system","content": "你是一个AI智能助手！"},
    { "role": "user","content": "你是谁"}
]
response = client.chat.completions.create(
    model="glm-4",
    messages=messages,
    stream=False,
    max_tokens=256,
    temperature=0.4,
    presence_penalty=1.2,
    top_p=0.8,
)
response

ChatCompletion(id='chatcmpl-AJ56zgVTbBQb47Seewbva8Ye3mmP6', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='我是一个名为 ChatGLM 的人工智能助手，是基于清华大学 KEG 实验室和智谱 AI 公司于2024共同训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。', role='assistant', function_call=None, tool_calls=None))], created=1722405795, model='glm-4', object='chat.completion', service_tier=None, system_fingerprint='fp_hawbL1lgv', usage=CompletionUsage(completion_tokens=45, prompt_tokens=25, total_tokens=70))

# 定义工具函数信息

In [9]:
def function_chat(use_stream=False):
    messages = [{
            "role": "user", "content": "What's the Celsius temperature in San Francisco?"
        },]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "format": {
                            "type": "string",
                            "enum": ["celsius", "fahrenheit"],
                            "description": "The temperature unit to use. Infer this from the users location.",
                        },
                    },
                    "required": ["location", "format"],
                },
            }
        },
    ]

    response = client.chat.completions.create(
        model="glm-4",
        messages=messages,
        tools=tools,
        stream=use_stream,
        max_tokens=256,
        temperature=0.9,
        presence_penalty=1.2,
        top_p=0.1,
        tool_choice="auto"
    )
    if response:
        if use_stream:
            for chunk in response:
                print(chunk)
        else:
            print(response)
    else:
        print("Error:", response.status_code)

# 调用工具函数
function_chat(use_stream=False)

ChatCompletion(id='chatcmpl-RYKvJIrYy5if1weSPDt07byh9Nzij', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_oS13YglBeAyqBUwutEVmyBNP', function=Function(arguments='{"location": "San Francisco, CA", "format": "celsius"}', name='get_current_weather'), type='function', index=0)]))], created=1722395612, model='glm-4', object='chat.completion', service_tier=None, system_fingerprint='fp_1Fk34RSST', usage=CompletionUsage(completion_tokens=21, prompt_tokens=222, total_tokens=243))
